# Project 1

This notebook shows a reference implementation of Project 1. For your group's submission, create your own implementation of the experiments below.

In [ ]:
# Run this cell to install the latest version of Qiskit Experiments
!pip install git+https://github.com/qiskit-community/qiskit-experiments.git

In [ ]:
import numpy as np
import qiskit as qs
import qiskit_ibm_runtime as ibm

import rustworkx as rx
import matplotlib.pyplot as plt

from qiskit_experiments.framework import ParallelExperiment

# Modules Needed for experiments
from qiskit_experiments.library import T1, T2Ramsey, T2Hahn, Tphi, StandardRB
from qiskit_experiments.library.randomized_benchmarking import LayerFidelity



In [ ]:
service = ibm.QiskitRuntimeService(name='rpi-quantum')
backend = service.backend('ibm_rensselaer')

twoq_gate = "ecr"
print(f"Device {backend.name} Loaded with {backend.num_qubits} qubits")
print(f"Two Qubit Gate: {twoq_gate}")

## $T_1$ Characterization

In [ ]:
t1_delays = np.linspace(0.0, 500.0, 26)*1e-6
t1_delays

In [ ]:
t1_exps = []
qubits = [28, 3, 102, 54, 44, 9]

for qubit in qubits:
    
    # Create the experiment for the qubit
    exp = T1(physical_qubits=(qubit,),
             delays=t1_delays,
             backend=backend,)
    
    # Add it to our list of experiments
    t1_exps.append(exp)

parallel_t1_exp = ParallelExperiment(t1_exps, backend=backend,)

In [ ]:
parallel_t1_exp.set_transpile_options(scheduling_method='asap',
                                      target=backend.target)

t1_data = parallel_t1_exp.run(backend=backend)

In [ ]:
t1_data.figure(2)

## $T_2$

In [ ]:
t2_delays = np.linspace(0.0, 250.0, 26)*1e-6
t2_delays

In [ ]:
t2_exps = []
qubits = [28, 3, 102, 54, 44, 9]

for qubit in qubits:
    
    # Create the experiment for the qubit
    exp = T2Hahn(physical_qubits=(qubit,),
                 delays=t2_delays,
                 backend=backend,)
    
    # Add it to our list of experiments
    t2_exps.append(exp)

parallel_t2_exp = ParallelExperiment(t2_exps, backend=backend,)

In [ ]:
parallel_t2_exp.set_transpile_options(scheduling_method='asap', target=backend.target)
t2_data = parallel_t2_exp.run(backend=backend)

## $T_2^*$

In [ ]:
t2star_exps = []
qubits = [28, 3, 102, 54, 44, 9]

for qubit in qubits:
    
    # Create the experiment for the qubit
    exp = T2Ramsey(physical_qubits=(qubit,),
                   delays=t2_delays,
                   backend=backend,
                  osc_freq=5e6)
    
    # Add it to our list of experiments
    t2star_exps.append(exp)

parallel_t2star_exp = ParallelExperiment(t2star_exps, backend=backend,)

In [ ]:
parallel_t2star_exp.set_transpile_options(scheduling_method='asap', target=backend.target)
t2star_data = parallel_t2star_exp.run(backend=backend)

In [ ]:
t2star_data.figure(0)

## Layer Fidelity

In [ ]:
num_qubits_in_chain = 100
coupling_map = backend.target.build_coupling_map(twoq_gate)
G = coupling_map.graph

In [ ]:
def to_edges(path):
    edges = []
    prev_node = None
    for node in path:
        if prev_node is not None:
            if G.has_edge(prev_node, node):
                edges.append((prev_node, node))
            else:
                edges.append((node, prev_node))
        prev_node = node
    return edges


def path_fidelity(path, correct_by_duration: bool = True, readout_scale: float = None):
    """Compute an estimate of the total fidelity of 2-qubit gates on a path.
    If `correct_by_duration` is true, each gate fidelity is worsen by
    scale = max_duration / duration, i.e. gate_fidelity^scale.
    If `readout_scale` > 0 is supplied, readout_fidelity^readout_scale
    for each qubit on the path is multiplied to the total fielity.
    The path is given in node indices form, e.g. [0, 1, 2].
    An external function `to_edges` is used to obtain edge list, e.g. [(0, 1), (1, 2)]."""
    path_edges = to_edges(path)
    max_duration = max(backend.target[twoq_gate][qs].duration for qs in path_edges)

    def gate_fidelity(qpair):
        duration = backend.target[twoq_gate][qpair].duration
        scale = max_duration / duration if correct_by_duration else 1.0
        # 1.25 = (d+1)/d) with d = 4
        return max(0.25, 1 - (1.25 * backend.target[twoq_gate][qpair].error)) ** scale

    def readout_fidelity(qubit):
        return max(0.25, 1 - backend.target["measure"][(qubit,)].error)

    total_fidelity = np.prod([gate_fidelity(qs) for qs in path_edges])
    if readout_scale:
        total_fidelity *= np.prod([readout_fidelity(q) for q in path]) ** readout_scale
    return total_fidelity


def flatten(paths, cutoff=None):  # cutoff not to make run time too large
    return [
        path
        for s, s_paths in paths.items()
        for t, st_paths in s_paths.items()
        for path in st_paths[:cutoff]
        if s < t
    ]

In [ ]:
paths = rx.all_pairs_all_simple_paths(
    G.to_undirected(multigraph=False),
    min_depth=num_qubits_in_chain,
    cutoff=num_qubits_in_chain,
)
paths = flatten(paths, cutoff=400)
if not paths:
    raise Exception(
        f"No qubit chain with length={num_qubits_in_chain} exists in {backend.name}. Try smaller num_qubits_in_chain."
    )

print(f"Selecting the best from {len(paths)} candidate paths (will take a few minutes)")
best_qubit_chain = max(paths, key=path_fidelity)
# from functools import partial
# best_qubit_chain = max(paths, key=partial(path_fidelity, correct_by_duration=True, readout_scale=1.0))
assert len(best_qubit_chain) == num_qubits_in_chain
print(f"Predicted LF from reported 2q-gate EPGs: {path_fidelity(best_qubit_chain)}")

In [ ]:
np.array(best_qubit_chain)

In [ ]:
# decompose the chain into two disjoint layers
all_pairs = to_edges(best_qubit_chain)
two_disjoint_layers = [all_pairs[0::2], all_pairs[1::2]]
two_disjoint_layers

In [ ]:
%%time
lfexp = LayerFidelity(
    physical_qubits=best_qubit_chain,
    two_qubit_layers=two_disjoint_layers,
    lengths=[2, 4, 8, 16, 24, 32, 48, 64, 72, 96, 128, 160, 192, 256, 320, 512],
    backend=backend,
    num_samples=32,
    seed=42,
    # two_qubit_gate="ecr",
    # one_qubit_basis_gates=["rz", "sx", "x"],
)


In [ ]:
# set maximum number of circuits per job to avoid errors due to too large payload
lfexp.experiment_options.max_circuits = 144

In [ ]:
print(f"Two Qubit Gate: {lfexp.experiment_options.two_qubit_gate}")
print(f"One Qubit Gate Set: {lfexp.experiment_options.one_qubit_basis_gates}")

In [ ]:
%%time
# look at one of the first three 2Q direct RB circuits quickly
circ_iter = lfexp.circuits_generator()
first_three_circuits = list(next(circ_iter) for _ in range(3))
first_three_circuits[1].draw(output="mpl",
                             style="clifford", 
                             idle_wires=False, 
                             fold=-1)

In [ ]:
%%time
# generate all circuits to run
circuits = lfexp.circuits()
print(f"{len(circuits)} circuits are generated.")

In [ ]:
%%time
# number of shots per job
nshots = 1024

# Run the LF experiment (generate circuits and submit the job)
exp_data = lfexp.run(shots=nshots)
# exp_data.auto_save = True
print(f"Run experiment: ID={exp_data.experiment_id} with jobs {exp_data.job_ids}]")

In [ ]:
df = exp_data.analysis_results(dataframe=True)

In [ ]:
df[(df.name == "ProcessFidelity")].head()

In [ ]:
for i in range(0, 5):
    display(exp_data.figure(i))


In [ ]:
pfdf = df[(df.name == "ProcessFidelity")]
pfdf[pfdf.value < 0.8]

In [ ]:
# find bad quality analysis results
pfdf[pfdf.quality == "bad"]

In [ ]:
# fill Process Fidelity values with zeros
pfdf = pfdf.fillna({"value": 0})

In [ ]:


# Compute LF by chain length assuming the first layer is full with 2q-gates
lf_sets, lf_qubits = two_disjoint_layers, best_qubit_chain
full_layer = [None] * (len(lf_sets[0]) + len(lf_sets[1]))
full_layer[::2] = lf_sets[0]
full_layer[1::2] = lf_sets[1]
full_layer = [(lf_qubits[0],)] + full_layer + [(lf_qubits[-1],)]

assert len(full_layer) == len(lf_qubits) + 1


In [ ]:
pfs = [pfdf.loc[pfdf[pfdf.qubits == qubits].index[0], "value"] for qubits in full_layer]
pfs = list(map(lambda x: x.n if x != 0 else 0, pfs))
pfs[0] = pfs[0] ** 2
pfs[-1] = pfs[-1] ** 2
np.array(pfs)

In [ ]:
job = service.job(exp_data.job_ids[0])
JOB_DATE = job.creation_date

In [ ]:
# Approximate 1Q RB fidelities at both ends by the square root of 2Q RB fidelity at both ends.
# For example, if we have [(0, 1), (1, 2), (2, 3), (3, 4)] 2Q RB fidelities and if we want to compute a layer fidelity for [1, 2, 3],
# we approximate the 1Q filedities for (1,) and (3,) by the square root of 2Q fidelities of (0, 1) and (3, 4).
chain_lens = list(range(4, len(pfs), 2))
chain_fids = []
for length in chain_lens:
    w = length + 1  # window size
    fid_w = max(
        np.sqrt(pfs[s]) * np.prod(pfs[s + 1 : s + w - 1]) * np.sqrt(pfs[s + w - 1])
        for s in range(len(pfs) - w + 1)
    )
    chain_fids.append(fid_w)

In [ ]:
# Plot LF by chain length
plt.title(f"Backend: {backend.name}, {JOB_DATE.strftime('%Y/%m/%d %H:%M')}")
plt.plot(
    chain_lens,
    chain_fids,
    marker="o",
    linestyle="-",
)
plt.xlim(0, chain_lens[-1] * 1.05)
plt.ylim(0.95 * min(chain_fids), 1)
plt.ylabel("Layer Fidelity")
plt.xlabel("Chain Length")
plt.grid()
plt.show()

In [ ]:
# Plot EPLG by chain length
num_2q_gates = [length - 1 for length in chain_lens]
chain_eplgs = [
    1 - (fid ** (1 / num_2q)) for num_2q, fid in zip(num_2q_gates, chain_fids)
]
plt.title(f"Backend: {backend.name}, {JOB_DATE.strftime('%Y/%m/%d %H:%M')}")
plt.plot(
    chain_lens,
    chain_eplgs,
    marker="o",
    linestyle="-",
)
plt.xlim(0, chain_lens[-1] * 1.05)
plt.ylabel("Error per Layered Gates")
plt.xlabel("Chain Length")
plt.grid()
plt.show()